In [36]:

import re
import pandas as pd
from Source import data_chanel_map
from Source import forward_color
from Source import reverse_color
from Source import reverse_map

class BinFile:
    def __init__(self, file) -> None:
        self.data = self.read_file(file)

    def read_file(self, file):
        with open(file, 'r') as f:
            return [line.strip('\n') for line in f.readlines()]

    def parser(self):
        lines = []
        for line in self.data:
            # print(line)
            if 'Version' in line:
                version = line.split('\t')[1]
            elif 'Chemistry' in line:
                kit_name = line.split('\t')[1]
            elif 'Panel Name' in line:
                panel_name = line.split('\t')[1]
            elif 'Marker Name' in line:
                marker_name = line.split('\t')[1]
            elif 'BinSet' in line:
                continue
            else:
                # print(panel_name)
                line_base = line.split('\t')
                line_base.insert(0, marker_name)
                line_base.insert(0, panel_name)
                line_base.insert(0, kit_name)
                line_base.insert(0, version)
                lines.append(line_base)
        df = pd.DataFrame(lines, columns=['version', 'package', 'panel', 'marker', 'base', 'min_bin', 'max_bin', 'color'])
        df = df[pd.notnull(df.color)]
        df['min_bin'] = round(df['min_bin'].astype('float'),2)
        df['max_bin'] = round(df['max_bin'].astype('float'),2)
        df['color'] = df['color'].apply(lambda x: x.lower())
        # conver yellow to black
        df['color'] = df['color'].apply(lambda x: 'black' if x  == 'yellow' else x)
        
        # update direction
        df['direction'] = df.apply(lambda row: self.update_direction(row['base'], row['color']), axis=1)


        # process duplicate
        # get max range file of bin setting if base duplicated
        df['binrange'] = df.max_bin - df.min_bin
        # df = (df.groupby(['version', 'package', 'panel', 'marker', 'base'], as_index=False)).sort_values('binrange', ascending=False)
        return df
    
    def update_direction(self, base, color):
        return 'forward' if forward_color.get(base) == color else 'reverse'


In [37]:
bin_file = './raw_data/panels/[2021.08 └┌╖ß] PGx-NP_PGx-NP_bins.txt'
bin = BinFile(bin_file)
data = bin.parser()

In [47]:
t = data.sort_values('binrange', ascending=False).groupby(['version', 'panel', 'marker', 'base', 'color'], as_index=False).first()
# t = 

In [48]:
data_c = t.groupby(['panel', 'marker', 'base'], as_index=False).size()

In [49]:
bin_file = './raw_data/panels/[2021.08 └┌╖ß] PGx-Premium_PGx-Premium_bins.txt'
bin = BinFile(bin_file)
data = bin.parser()

## update from laptop

In [86]:
# data.to_excel('Bin_setting_table.xlsx', index=False)

In [ ]:
    # def pattern_location(self, data, pattern = 'Panel Name'):
    #     return [i for i,line in enumerate(data) if pattern in line]

    # def name_extract(self, line):
    #     return line.split('\t')[1]

    # def line_extract(self, data, start, stop):
    #     return data[start+1:stop]

    # def range_generate(self, lst, n=2):
    #     return [lst[i:i+n] for i in range(len(lst)-n+1)]

    # def marker_frame(self, data, start, stop):
    #     marker_name = self.name_extract(data[0])
    #     print(data[0])
    #     markers = [m.split('\t') for m in data[1:]]

    #     df = pd.DataFrame(markers, columns=['base', 'min_bin', 'max_bin', 'color'])
    #     df['marker'] = marker_name

    #     return df

    # def panel_frame(self, start, stop):

    #     panel_df = pd.DataFrame()
    #     panel_data = self.line_extract(self.data, start, stop)
    #     marker_loc = self.pattern_location(panel_data, "Marker Name")
    #     marker_loc.append(len(panel_data))
    #     # print(marker_loc)

    #     for m_start, m_stop in self.range_generate(marker_loc):
    #         marker_data = self.line_extract(panel_data, m_start, m_stop)
    #         # print(marker_data)
    #         # print(m_start, m_stop, len(marker_data))
    #         marker_df = self.marker_frame(marker_data, m_start, m_stop)
    #         panel_df = pd.concat([panel_df, marker_df])
        
    #     return panel_df


